In [1]:
from PyPDF2 import PdfReader
import re
import openai
from pdf2image import convert_from_bytes
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())

# https://stackoverflow.com/questions/43314559/extracting-text-from-a-pdf-all-pages-and-output-file-using-python
pdf_file = 'Badreesh_Shetty_Resume.pdf'


In [2]:
def read_pdf_page(file, page_number):
    pdfReader = PdfReader(file)
    page = pdfReader.pages[page_number]
    return page.extract_text()

def clean_resume(pdf_str):
    applicant_resume = re.sub('\s[,.]', ',', pdf_str)
    applicant_resume = re.sub('[\n]+', '\n', applicant_resume)
    applicant_resume = re.sub('[\s]+', ' ', applicant_resume)

    return applicant_resume


def get_response(prompt, system = '', temperature = 0.5):
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [{"role": "user", "content": prompt},
                   {"role": "system", "content": system}],
        temperature = temperature
    ) 
    return response['choices'][0]['message']['content']

In [3]:
pdfReader = PdfReader(pdf_file)
page_numbers = list(range(1, len(pdfReader.pages)+1))
# selected_page = st.selectbox("Select a page", page_numbers)
selected_page = 0


pdf_str=read_pdf_page(pdf_file, selected_page)

applicant_resume=clean_resume(pdf_str)

In [4]:
# set a system prompt
system_prompt = '''You are a helpful AI Job assistant, provide the answer in a json format.'''

In [5]:
message_prompt =str(applicant_resume)+'''
Summarize the text below into a JSON with exactly the following structure 
{basic_info: {first_name, last_name, full_name, email, phone_number, portfolio_website_url, 
linkedin_url, github_main_page_url, university, education_level (BS, MS, or PhD), graduation_year, 
graduation_month, majors, GPA, total_years_experience_with_months}, 
work_experience: [{job_title, company, location, duration, job_summary}], 
project_experience:[{project_name, project_description}], 
technical_skills:[elaborate_skills_mentioned_throughout_in_resume],
intrapersonal_skills: if intrapersonal skills then [elaborate_skills_mentioned_throughout_in_resume] 
else null}.'''
 
## This code will send the request and display the response
gpt_response_resume = get_response(message_prompt,system_prompt)
print(gpt_response_resume)

{
  "basic_info": {
    "first_name": "Badreesh",
    "last_name": "Shetty",
    "full_name": "Badreesh Shetty",
    "email": "badreeshshetty@gmail.com",
    "phone_number": "617-971-7369",
    "portfolio_website_url": "badreeshshetty.github.io",
    "linkedin_url": "linkedin.com/in/badreesh-shetty",
    "github_main_page_url": "badreeshshetty.github.io",
    "university": "Northeastern University",
    "education_level": "Master's",
    "graduation_year": 2023,
    "graduation_month": "December",
    "majors": "Business Analytics",
    "GPA": null,
    "total_years_experience_with_months": "3 years and 5 months"
  },
  "work_experience": [
    {
      "job_title": "Data Scientist",
      "company": "Centelon",
      "location": "Mumbai, India",
      "duration": "01/2020 - 05/2022",
      "job_summary": "Executed an AWS Python-based ETL pipeline for a real-time OCR tool, handling 30,000+ financial documents (100 GB+) from email attachments for a leading pharmaceutical client. Realized

In [6]:
import json
gpt_response_resume_json = json.loads(gpt_response_resume)
# print(gpt_response_resume_json)
gpt_response_resume_json['basic_info']

{'first_name': 'Badreesh',
 'last_name': 'Shetty',
 'full_name': 'Badreesh Shetty',
 'email': 'badreeshshetty@gmail.com',
 'phone_number': '617-971-7369',
 'portfolio_website_url': 'badreeshshetty.github.io',
 'linkedin_url': 'linkedin.com/in/badreesh-shetty',
 'github_main_page_url': 'badreeshshetty.github.io',
 'university': 'Northeastern University',
 'education_level': "Master's",
 'graduation_year': 2023,
 'graduation_month': 'December',
 'majors': 'Business Analytics',
 'GPA': None,
 'total_years_experience_with_months': '3 years and 5 months'}

In [7]:
 for n,experience in enumerate(gpt_response_resume_json['work_experience']):
        if gpt_response_resume_json['work_experience'][n]:
            print("Job Title:", experience["job_title"])
            print("Company:", experience["company"])
            print("Location:", experience["location"])
            print("Duration:", experience["duration"])
            print("Job Summary:", experience["job_summary"])
            print("\n")

Job Title: Data Scientist
Company: Centelon
Location: Mumbai, India
Duration: 01/2020 - 05/2022
Job Summary: Executed an AWS Python-based ETL pipeline for a real-time OCR tool, handling 30,000+ financial documents (100 GB+) from email attachments for a leading pharmaceutical client. Realized a 48% decrease in manual effort, resulting in a monthly time savings of 50 hours. Led a team of 3 data scientists in conceptualization and execution of a fuzzy string-matching algorithm that improved accuracy and speed of key information extraction resulting in a 17% decrease in processing time. Built the backend infrastructure using Django REST API framework and SQL which facilitated client's access to a CRUD web-app functionality through monthly reports, delivering valuable analytics for data-driven decision-making. Utilized Plotly-Dash and Excel to create interactive visualizations and dashboards, improving stakeholders' comprehension of key findings. Collaborated with cross-functional teams to 

In [8]:
for experience in gpt_response_resume_json['work_experience']:
    print("Job Title:", experience["job_title"])
    print("Company:", experience["company"])
    print("Location:", experience["location"])
    print("Duration:", experience["duration"])
    print("Job Summary:", experience["job_summary"])
    print("\n")

Job Title: Data Scientist
Company: Centelon
Location: Mumbai, India
Duration: 01/2020 - 05/2022
Job Summary: Executed an AWS Python-based ETL pipeline for a real-time OCR tool, handling 30,000+ financial documents (100 GB+) from email attachments for a leading pharmaceutical client. Realized a 48% decrease in manual effort, resulting in a monthly time savings of 50 hours. Led a team of 3 data scientists in conceptualization and execution of a fuzzy string-matching algorithm that improved accuracy and speed of key information extraction resulting in a 17% decrease in processing time. Built the backend infrastructure using Django REST API framework and SQL which facilitated client's access to a CRUD web-app functionality through monthly reports, delivering valuable analytics for data-driven decision-making. Utilized Plotly-Dash and Excel to create interactive visualizations and dashboards, improving stakeholders' comprehension of key findings. Collaborated with cross-functional teams to 

In [9]:
# import json
# try:
#     parsed_response = json.loads(gpt_response)
#     print(json.dumps(parsed_response, indent=2))  # Display the formatted JSON
# except json.JSONDecodeError as e:
#     print(f"Error decoding JSON: {e}")

In [10]:
job_description='''
About the job
Cox Communications is seeking a Senior Data Scientist for the Analytics + Intelligence Team (A+I). This role will report to a Manager of Data Science and will be responsible for architecture and development of analytics applications developed by the Operations & Technology Analytics team. This role will play a central role in the ongoing design and development of key decision-making analytics applications that are deeply integrated into both customer facing and employee facing platforms throughout operations.

What You'll Do

 Analytics Application Architecture:  Identify or design repeatable patterns for implementing analytics application use cases.

 Plan and document key logic paths, service architecture, and data stores for new use cases
 Review of existing services for maintainability, scalability, and opportunities for complexity reduction
 Design with a bias towards observability, reusability, testability, and scale
 Design of test plans with a preference for automation where possible

 Development  : Develop analytics applications using industry best practices for design, development, and testing. Make recommendations where needed for tools, services, and ways of working.

 Minimum 

 Bachelor's degree in a related discipline and 4 years' experience in a related field. The right candidate could also have a different combination, such as a master's degree and 2 years' experience; a Ph.D. and up to 1 year of experience; or 16 years' experience in a related field
 Strong programming skills and ability to utilize a variety of data/analytic/software languages and tools. (ex: Python, SQL, Docker, Java, etc.)
 Experience with analytics application design and development.
 Good communication skills. The ability to comprehend and communicate advanced technical topics to non-technical business partners.
 Experience with source code version control software, like Git or Bitbucket, and CICD pipelines like Jenkins.
 Experience designing and implementing large-scale, high data or user volume solutions in cloud platforms such as Google, Microsoft, or Amazon.

Preferred

 Degree in computer science, engineering or closely related field.
 Knowledge and experience with AWS Services including Lambda, Step function, CloudWatch, EventBridge, EC2, EMR, RedShift, SQS, RDS, IAM, API Gateway.
 Experience within the telecommunications industry, cable industry or consulting.
 Experience with data visualization solutions and data visualization tools.
 Experience with microservice development (Docker, Kubernetes , etc.).
 Experience in automated test design and implementation
 Experience in data store selection for large-scale applications (high data volume and/or high read/write)

About Cox Communications

Cox Communications is the largest private telecom company in America, serving six million homes and businesses. That's a lot, but we also proudly serve our employees. Our benefits and our award-winning culture are just two of the things that make Cox a coveted place to work. If you're interested in bringing people closer through broadband, smart home tech and more, join Cox Communications today!

About Cox

Cox empowers employees to build a better future and has been doing so for over 120 years. With exciting investments and innovations across transportation, communications, cleantech and healthcare, our family of businesses - which includes Cox Automotive and Cox Communications - is forging a better future for us all. Ready to make your mark? Join us today!

Benefits of working at Cox may include health care insurance (medical, dental, vision), retirement planning (401(k)), and paid days off (sick leave, parental leave, flexible vacation/wellness days, and/or PTO). For more details on what benefits you may be offered, visit our benefits page .

Cox is an Equal Employment Opportunity employer - All qualified applicants/employees will receive consideration for employment without regard to that individual's age, race, color, religion or creed, national origin or ancestry, sex (including pregnancy), sexual orientation, gender, gender identity, physical or mental disability, veteran status, genetic information, ethnicity, citizenship, or any other characteristic protected by law.

Statement to ALL Third-Party Agencies and Similar Organizations: Cox accepts resumes only from agencies with which we formally engage their services. Please do not forward resumes to our applicant tracking system, Cox employees, Cox hiring manager, or send to any Cox facility. Cox is not responsible for any fees or charges associated with unsolicited resumes.

'''

In [11]:
message_prompt =str(job_description)+'''
Summarize the text below into a JSON with exactly the following structure 
{basic_info: {company_name, job_role, required_experience_for_the_role,qualifications_for_the_role}},
technical_skills:[skills_mentioned_in_the_job_application],
intrapersonal_skills: if intrapersonal skills then [skills_mentioned_in_the_job_application] 
else null}.'''

In [12]:
## This code will send the request and display the response
gpt_response_jd = get_response(message_prompt,system_prompt)
print(gpt_response_jd)

{
  "basic_info": {
    "company_name": "Cox Communications",
    "job_role": "Senior Data Scientist",
    "required_experience_for_the_role": "Bachelor's degree in a related discipline and 4 years' experience in a related field. The right candidate could also have a different combination, such as a master's degree and 2 years' experience; a Ph.D. and up to 1 year of experience; or 16 years' experience in a related field",
    "qualifications_for_the_role": "Strong programming skills and ability to utilize a variety of data/analytic/software languages and tools. (ex: Python, SQL, Docker, Java, etc.), experience with analytics application design and development, good communication skills, experience with source code version control software, experience designing and implementing large-scale, high data or user volume solutions in cloud platforms"
  },
  "technical_skills": [
    "Python",
    "SQL",
    "Docker",
    "Java",
    "Git",
    "Bitbucket",
    "Jenkins",
    "Google Cloud",


In [14]:
gpt_response_resume_json['technical_skills']

['Python',
 'R',
 'Django',
 'Flask',
 'FastAPI',
 'PostgreSQL',
 'MongoDB',
 'Redis',
 'Snowflake',
 'AWS',
 'Alteryx',
 'Databricks',
 'Airflow',
 'Kafka',
 'Cassandra',
 'Airbyte',
 'Redshift',
 'BigQuery',
 'Hive',
 'PowerBI',
 'Pandas',
 'Keras',
 'Scikit-Learn',
 'Pytorch',
 'Pyspark',
 'Matplotlib',
 'Seaborn',
 'Docker',
 'Streamlit',
 'DBT',
 'Cron',
 'PCA',
 'Kmeans Clustering',
 'Naive Bayes',
 'Decision Tree',
 'Xgboost',
 'CNN',
 'RNN']

In [15]:
message_prompt ='''
Based on the technical skills mentioned on the resume here:'''+str(gpt_response_resume)+''' and technical skills \
mentioned on the job description here:'''+str(gpt_response_jd)+''' Create a 100 score around where skills 
where present in both are shown and not present in job description is shown'''

In [16]:
comparison_tech_skills=get_response(message_prompt,system_prompt)

In [17]:
print(comparison_tech_skills)

{
  "skills_match_score": 100,
  "matched_skills": [
    "Python",
    "SQL",
    "Docker"
  ],
  "unmatched_skills": [
    "R",
    "Django",
    "Flask",
    "FastAPI",
    "PostgreSQL",
    "MongoDB",
    "Redis",
    "Snowflake",
    "AWS",
    "Alteryx",
    "Databricks",
    "Airflow",
    "Kafka",
    "Cassandra",
    "Airbyte",
    "Redshift",
    "BigQuery",
    "Hive",
    "PowerBI",
    "Pandas",
    "Keras",
    "Scikit-Learn",
    "Pytorch",
    "Pyspark",
    "Matplotlib",
    "Seaborn",
    "Streamlit",
    "DBT",
    "Cron",
    "PCA",
    "Kmeans Clustering",
    "Naive Bayes",
    "Decision Tree",
    "Xgboost",
    "CNN",
    "RNN"
  ]
}
